5 - Email analysis:
Below are code examples that will help you parse the single email message file, or whole directory with malicous emails and exctracts various medatata for further use.
It can be combined with previous codes e.g to exctrat named entities, sentiment or even draw a graph.

Directory and exmaple file selection:

In [2]:
#----
directory="./emails/"
filename=directory+'1.eml'
#----

Working example for mailparser library, that you can use for your for further work:


In [3]:
import mailparser

#https://pypi.org/project/mail-parser/
"""
mail = mailparser.parse_from_bytes(byte_mail)
mail = mailparser.parse_from_file(f)
mail = mailparser.parse_from_file_msg(outlook_mail)
mail = mailparser.parse_from_file_obj(fp)
mail = mailparser.parse_from_string(raw_mail)

mail.attachments: list of all attachments
mail.body
mail.date: datetime object in UTC
mail.defects: defect RFC not compliance
mail.defects_categories: only defects categories
mail.delivered_to
mail.from_
mail.get_server_ipaddress(trust="my_server_mail_trust")
mail.headers
mail.mail: tokenized mail in a object
mail.message: email.message.Message object
mail.message_as_string: message as string
mail.message_id
mail.received
mail.subject
mail.text_plain: only text plain mail parts in a list
mail.text_html: only text html mail parts in a list
mail.text_not_managed: all not managed text (check the warning logs to find content subtype)
mail.to
mail.to_domains
mail.timezone: returns the timezone, offset from UTC
mail.mail_partial: returns only the mains parts of emails
    
#Write attachments on disc
mail.write_attachments(base_path)
"""


raw_email=filename

mail = mailparser.parse_from_file(raw_email)

print("[1]",mail.from_,type(mail.from_))
print("[2]",mail.defects)
print("[3]",mail.defects_categories)
print("[4]",mail.to)
print("[5]",mail.to_domains)
print("[6]",mail.received)
print("[7]",mail.get_server_ipaddress(trust="my_server_mail_trust"))
print("[8]",mail.delivered_to)
print("[9]",mail.attachments)
print("[10]",mail.subject)
print("[11]",mail.text_not_managed)
#print("[12]",mail.headers)
#print("[13]",mail.mail_partial)
#print("[14]",mail.text_plain) #works
#print("[15]",mail.text_html) #works
print("[16]",mail.message_id)
#print("[17]",mail.message_as_string)
#print("[18]",mail.body)
print("[19]",mail.timezone)

[1] [('Оргкомитет', 'news@hrsummit.ru')] <class 'list'>
[2] []
[3] set()
[4] [('', 'karasek.jindrich@gmail.com')]
[5] ['gmail.com']
[6] [{'by': 'smtp368.emlone.com', 'id': 'hed7s62erm86', 'for': '<karasek.jindrich@gmail.com>', 'envelope_from': 'postman3496981@justeml.com', 'date': 'Tue, 15 Nov 2022 08:37:23 +0000 envelope-from <postman3496981@justeml.com>', 'hop': 1, 'date_utc': '2022-11-15T08:37:23', 'delay': 0}, {'from': 'smtp573.emlone.com smtp573.emlone.com. 87.246.187.152', 'by': 'mx.google.com', 'with': 'ESMTPS', 'id': 'i1-20020a2ea361000000b0026fbd8bb585si5659533ljn.64.2022.11.15.00.37.23', 'for': '<karasek.jindrich@gmail.com> version=TLS1_2 cipher=ECDHE-ECDSA-AES128-GCM-SHA256 bits=128/128', 'date': 'Tue, 15 Nov 2022 00:37:23 -0800 PST', 'hop': 2, 'date_utc': '2022-11-15T08:37:23', 'delay': 0.0}, {'by': '2002:a05:6638:59c:b0:375:9b14:11af', 'with': 'SMTP', 'id': 'a28csp3262577jar', 'date': 'Tue, 15 Nov 2022 00:37:24 -0800 PST', 'hop': 3, 'date_utc': '2022-11-15T08:37:24', 'dela

Python functions ready to be incorporated to the analytical framework:
"email_miner_text" is applied to each email file to extract the metadata for the hunting.
"email_dir_miner" is an alternative of above function, in case you have already a pile of eml. files to process and also generates the graph of sender- receiver structure to identify the important topological structures in chain mail campaigns.

In [4]:
def email_miner_text(email_file): # takes single eml file and mines the metadata for the analysis.
    import datetime
    import json
    import eml_parser
    import numpy as np
    import pandas as pd
    import os
    import mailparser
    #https://pypi.org/project/mail-parser/
    """Dataframe design"""
    lcolumns=["Filename","messageID","from","subject","mailto","mailtod","received","attachments","text-html","text"]
    ledf=pd.DataFrame(columns=lcolumns)
    try:
        with open(email_file,'rb') as fhdl:
            raw_email=fhdl.read()
            #print(filename)
            #all the code for edf filling comes here
            mail = mailparser.parse_from_file(raw_email)
            data=np.array([str(file),str(mail.message_id),str(mail.from_),str(mail.subject),str(mail.to),str(mail.to_domains),str(mail.received),str(mail.attachments),str(mail.text_html),str(mail.text)])
            #row=pd.Series(data,index=lcolumns)
            #ledf=ledf.append(pd.Series(row),ignore_index=True)
    except:
        pass
    #print(ledf)
    #print(ledf.describe())
    #print(ledf["subject"],ledf["Filename"])
    #deduplicate the edgelist
    #ledf.drop_duplicates(keep='first')
    #save the edgelists to a csv files
    #ledf.to_csv("/Users/jindrich_karasek/data-science/disinfo-corpus/maily/ledf.csv", sep=',', encoding='utf-8')
    #print(mail.message_as_string)
    #print(mail.text_plain)
    return mail.text_plain

def email_dir_miner(path): #Generates the matrix for the graph of the sending emails structure.
    """Read ALL directory,TOP DOWN from starting point, select all the *.eml files and parse metadata into the dataframe"""
    import datetime
    import json
    import eml_parser
    import numpy as np
    import pandas as pd
    import os
    import mailparser
    import tqdm

    #https://pypi.org/project/mail-parser/
    """Dataframe design"""
    #lcolumns=["Filename","messageID","from","subject","mailto","mailtod","received","attachments","text-html","text"]
    lcolumns=["from","mailto"]
    ledf=pd.DataFrame(columns=lcolumns)
    """iterate over the files and add pd series generated from each into the final datatframe"""
    #https://www.bogotobogo.com/python/python_traversing_directory_tree_recursively_os_walk.php
    
    i=0
    for pth,subdirs,files in os.walk(path):
        for name in files:
            file=os.path.join(pth,name)
            #print(file)
            filename=str(str(path)+(os.fsdecode(file)))
            #print(file)
            if file.endswith(".eml"):
                i=i+1
                #print(i)
                try:
                    with open(file,'rb') as fhdl:
                        raw_email=fhdl.read()
                        #print(filename)
                        #all the code for edf filling comes here
                        mail = mailparser.parse_from_bytes(raw_email)
                        #data=np.array([str(file),str(mail.message_id),str(mail.from_),str(mail.subject),str(mail.to),str(mail.to_domains),str(mail.received),str(mail.attachments),str(mail.text_html)])
                        data={"Filename":str(file),"messageID":str(mail.message_id),"from":str(mail.from_),"subject":str(mail.subject),"mailto":str(mail.to),"mailtod":str(mail.to_domains),"received":str(mail.received),"attachments":str(mail.attachments),"text-html":str(mail.text_html),"text":str(mail.text)}
                        #data={"from":str(mail.from_).replace('[','').replace(']','').replace('(','').replace(')',''),"mailto":str(mail.to).replace(']','').replace('[','').replace('(','').replace(')','')}
                        row=pd.Series(data,index=lcolumns)
                        ledf=ledf.append(pd.Series(row),ignore_index=True)
                        continue
                except:
                    continue    
            else:
                continue
        if i > 10000000:
            break
        
    #Turning ledf into the dataframe-list of emails:
    email_df1=ledf["from"]
    email_df2=ledf["mailto"]
    
    email_df=email_df1.append(email_df2)
                          
    #deduplicate the edgelist
    ledf.drop_duplicates(keep='first')
    email_df.drop_duplicates(keep='first')
    print("[*]Duplicates dropped.")
    #print(edf)
    print(ledf.describe())
    #print(ledf["subject"],ledf["from"])
    #save the edgelists to a csv files
    ledf_dir=directory+"edf.csv"
    ledf.to_csv(ledf_dir, sep=',', encoding='utf-8')
    ledf_dir2=directory+"email_df.csv"
    email_df.to_csv(ledf_dir2,sep=',',index=False,header=False)
    print("[*] CSV file saved, done!")

#------------------------------

email_dir_miner("./emails/")
#print("Output of email_miner_text(): ","\n",email_miner_text(raw_email))
#print("Output of email_miner_data(): ","\n",email_miner_data(raw_email))

[*]Duplicates dropped.
                                                     from  \
count                                                  20   
unique                                                 16   
top     [('LOGYTalks - Blockchain & Cryptocurrency Sum...   
freq                                                    2   

                                      mailto  
count                                     20  
unique                                     9  
top     [('', 'karasek.jindrich@gmail.com')]  
freq                                       8  
[*] CSV file saved, done!
